In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
%matplotlib inline
%reload_kedro
%load_ext nb_black

2022-03-28 10:57:21,828 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-03-28 10:57:21,986 - root - INFO - ** Kedro project transmi
2022-03-28 10:57:21,988 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-03-28 10:57:22,153 - blib2to3.pgen2.driver - INFO - Generating grammar tables from /opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/blib2to3/Grammar.txt
2022-03-28 10:57:22,171 - blib2to3.pgen2.driver - INFO - Writing grammar tables to /Users/juanazuero/Library/Caches/black/22.1.0/Grammar3.8.0.final.0.pickle
2022-03-28 10:57:22,173 - blib2to3.pgen2.driver - INFO - Writing failed: [Errno 2] No such file or directory: '/Users/juanazuero/Library/Caches/black/22.1.0/tmp1fpbupa3'
2022-03-28 10:57:22,175 - blib2to3.pgen2.driver - INFO - Generating grammar tables from /opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/blib2to3/PatternGrammar.txt
2022-03-28 10

<IPython.core.display.Javascript object>